In [6]:
import torch
from torch import nn
import torch.nn.functional as F
import lightning as L
import torchmetrics

from torchvision import transforms
from torchvision.datasets import Imagenette
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint

In [7]:



# Prepare the dataset
train_transforms = transforms.Compose([
    # transforms.RandomHorizontalFlip(),
    # transforms.RandomCrop(160, padding=8, padding_mode='reflect'),  # Data Augmentation
    transforms.CenterCrop(160),
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
    transforms.Grayscale()
])

test_transforms = transforms.Compose([
    transforms.CenterCrop(160),
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
    transforms.Grayscale()
])

train_dataset = Imagenette("data/imagenette/train/", split="train", size="160px",  transform=train_transforms)

# Use 10% of the training set for validation
train_set_size = int(len(train_dataset) * 0.9)
val_set_size = len(train_dataset) - train_set_size

seed = torch.Generator().manual_seed(42)
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_set_size, val_set_size], generator=seed)
val_dataset.dataset.transform = test_transforms

# Use DataLoader to load the dataset
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, num_workers=8, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=128, num_workers=8, shuffle=False)

# Configure the test dataset
test_dataset = Imagenette("data/imagenette/test/", split="val", size="160px",  transform=test_transforms)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, num_workers=8, shuffle=False)

# Add EarlyStopping
early_stop_callback = EarlyStopping(monitor="val_loss",
                                    mode="min",
                                    patience=5)

# Configure Checkpoints
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    mode="min"
)

In [4]:
# for images, labels in train_loader:
#   # Get the shape of the first image in the batch
#   image_shape = images[0].shape
#   print(f"Image shape: {image_shape}")
#   break  # You only need to check the shape once

## Experiment -0

In [8]:
class BaselineModel(L.LightningModule):
    def __init__(self, num_classes=10):
        super().__init__()

        self.estimator = nn.Sequential(
            nn.Linear(64 * 64, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)

        self.train_loss=[]
        self.val_loss=[]

    def forward(self, x):
        x = x.view(x.shape[0], -1)

        return self.estimator(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        self.log("train_loss", loss,prog_bar=True)
        self.train_loss.append(loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        self.accuracy(y_hat, y)

        self.val_loss.append(loss)


        self.log("val_accuracy", self.accuracy,prog_bar=True)
        self.log("val_loss", loss,prog_bar=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        self.accuracy(y_hat, y)

        self.log("test_accuracy", self.accuracy)
        self.log("test_loss", loss)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [4]:
model = BaselineModel()

# Add EarlyStopping
early_stop_callback = EarlyStopping(monitor="val_loss",
                                    mode="min",
                                    patience=5)

# Configure Checkpoints
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    mode="min"
)

# Fit the model
trainer = L.Trainer(callbacks=[early_stop_callback, checkpoint_callback])
trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
d:\anaconda3\envs\cse6363\Lib\site-packages\lightning\pytorch\trainer\connectors\logger_connector\logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
d:\anaconda3\envs\cse6363\Lib\site-packages\lightning\pytorch\loops\utilities.py:72: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   

Epoch 8: 100%|██████████| 67/67 [01:09<00:00,  1.04s/it, v_num=20, train_loss=0.813, val_accuracy=0.348, val_loss=2.370]


In [5]:
print(f"Best model stored at {trainer.checkpoint_callback.best_model_path}")

## Restoring the best checkpoint
model = BaselineModel.load_from_checkpoint(
            trainer.checkpoint_callback.best_model_path
        )
##Obtaining the test metrics
test_result = trainer.test(model, dataloaders=test_loader, verbose=False)
print(f"Train Test result: {test_result}")

print(f"Best validation loss {trainer.checkpoint_callback.best_model_score.item()}\nTest loss: {test_result[0]['test_loss']}\nTest Accuracy: {test_result[0]['test_accuracy']}")

Best model stored at d:\UTA\Sem-2\ML\CSE6363\assignments\assignment4\lightning_logs\version_20\checkpoints\epoch=3-step=268.ckpt


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 31/31 [00:03<00:00,  8.39it/s]
Train Test result: [{'test_accuracy': 0.3314649760723114, 'test_loss': 1.9839969873428345}]
Best validation loss 1.9318835735321045
Test loss: 1.9839969873428345
Test Accuracy: 0.3314649760723114


## Experiment-1

In [36]:
class CNNModel(BaselineModel):
    def __init__(self, num_classes=10):
        super().__init__()

        self.estimator = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),  # Input channel = 1 for grayscale
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),  # Flatten output for fully connected layers
            nn.Linear(64 * 16 * 16, 128),  
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)

    def forward(self, x):
        
        x = x.view(x.shape[0], 1, 64, 64)  # Reshape for CNN (1 channel for grayscale)
        
        return self.estimator(x)

In [5]:
model = CNNModel()

# Add EarlyStopping
early_stop_callback = EarlyStopping(monitor="val_loss",
                                    mode="min",
                                    patience=5)

# Configure Checkpoints
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    mode="min"
)

# Fit the model
trainer = L.Trainer(callbacks=[early_stop_callback, checkpoint_callback])
trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
d:\anaconda3\envs\cse6363\Lib\site-packages\lightning\pytorch\trainer\connectors\logger_connector\logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
d:\anaconda3\envs\cse6363\Lib\site-packages\lightning\pytorch\loops\utilities.py:72: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   

Epoch 9: 100%|██████████| 67/67 [01:07<00:00,  1.00s/it, v_num=11]         


In [6]:
## Restoring the best checkpoint
model = CNNModel.load_from_checkpoint(
            trainer.checkpoint_callback.best_model_path
        )
##Obtaining the test metrics
test_result = trainer.test(model, dataloaders=test_loader, verbose=False)
print(f"Train Test result: {test_result}")

print(f"Best validation loss {trainer.checkpoint_callback.best_model_score.item()}\nTest loss: {test_result[0]['test_loss']}\nTest Accuracy: {test_result[0]['test_accuracy']}")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 31/31 [00:01<00:00, 26.95it/s]
Train Test result: [{'test_accuracy': 0.571974515914917, 'test_loss': 1.3563098907470703}]
Best validation loss 1.3125014305114746
Test loss: 1.3563098907470703
Test Accuracy: 0.571974515914917


## Experiment-2

In [37]:
class AllConvNet(BaselineModel):
    def __init__(self, num_classes=10):
        super().__init__()

        self.estimator = nn.Sequential(
            nn.Conv2d(1, 96, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(96, 96, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(96, 96, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(96, 192, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(192, 192, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(192, 192, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(192, 192, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(192, 192, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(192, 10, kernel_size=1),
            nn.ReLU(),
        )
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.softmax = nn.Softmax(dim=1)

        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)

    def forward(self, x):
        
        x = self.estimator(x)
        x = self.avg_pool(x)
        x = torch.flatten(x, 1)
        x = self.softmax(x)
        return x

In [7]:
model = AllConvNet()

# Add EarlyStopping
early_stop_callback = EarlyStopping(monitor="val_loss",
                                    mode="min",
                                    patience=5)

# Configure Checkpoints
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    mode="min"
)

# Fit the model
trainer = L.Trainer(callbacks=[early_stop_callback, checkpoint_callback])
trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type               | Params
-------------------------------------------------
0 | estimator | Sequential         | 1.4 M 
1 | accuracy  | MulticlassAccuracy | 0     
2 | avg_pool  | AdaptiveAvgPool2d  | 0     
3 | softmax   | Softmax            | 0     
-------------------------------------------------
1.4 M     Trainable params
0         Non-trainable params
1.4 M     Total params
5.472     Total estimated model params size (MB)


Epoch 39: 100%|██████████| 67/67 [01:19<00:00,  1.19s/it, v_num=12]        


In [9]:
## Restoring the best checkpoint
model = AllConvNet.load_from_checkpoint(
            trainer.checkpoint_callback.best_model_path
        )
##Obtaining the test metrics
test_result = trainer.test(model, dataloaders=test_loader, verbose=False)

print(f"Best validation loss {trainer.checkpoint_callback.best_model_score.item()}\nTest loss: {test_result[0]['test_loss']}\nTest Accuracy: {test_result[0]['test_accuracy']}")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 31/31 [00:02<00:00, 13.66it/s]
Best validation loss 1.9683494567871094
Test loss: 1.9843178987503052
Test Accuracy: 0.4736305773258209


## Experiment-3

In [38]:
class CNNModelwithDropOut(BaselineModel):
    def __init__(self, num_classes=10,img_shape=(1,64,64)):
        super().__init__()

        self.estimator = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),  # Input channel = 1 for grayscale
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(p=0.3),  # Add dropout after max pooling
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(p=0.3),  # Add dropout after max pooling
            nn.Flatten(),  # Flatten output for fully connected layers
            nn.Linear(64 * 16 * 16, 128),  
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)
        self.img_shape = img_shape

    def forward(self, x):
        
        x = x.view(x.shape[0], self.img_shape[0], self.img_shape[1], self.img_shape[2])  # Reshape for CNN (1 channel for grayscale)
        
        return self.estimator(x)

In [22]:
model = CNNModelwithDropOut()

# Add EarlyStopping
early_stop_callback = EarlyStopping(monitor="val_loss",
                                    mode="min",
                                    patience=5)

# Configure Checkpoints
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    mode="min"
)

# Fit the model
trainer = L.Trainer(callbacks=[early_stop_callback, checkpoint_callback])
trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
d:\anaconda3\envs\cse6363\Lib\site-packages\lightning\pytorch\loops\utilities.py:72: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type               | Params
-------------------------------------------------
0 | estimator | Sequential         | 2.1 M 
1 | accuracy  | MulticlassAccuracy | 0     
-------------------------------------------------
2.1 M     Trainable params
0         Non-trainable params
2.1 M     Total params
8.470     Total estimated model params size (MB)


Epoch 13: 100%|██████████| 67/67 [01:05<00:00,  1.02it/s, v_num=15]        


In [23]:
## Restoring the best checkpoint
print(f"Restoring from {trainer.checkpoint_callback.best_model_path}")

model = CNNModelwithDropOut.load_from_checkpoint(
            trainer.checkpoint_callback.best_model_path
        )
##Obtaining the test metrics
test_result = trainer.test(model, dataloaders=test_loader, verbose=False)

print(f"Best validation loss {trainer.checkpoint_callback.best_model_score.item()}\nTest loss: {test_result[0]['test_loss']}\nTest Accuracy: {test_result[0]['test_accuracy']}")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 31/31 [00:01<00:00, 26.09it/s]
Train Test result: [{'test_accuracy': 0.5467516183853149, 'test_loss': 1.3912665843963623}]
Best validation loss 1.3289514780044556
Test loss: 1.3912665843963623
Test Accuracy: 0.5467516183853149


## Experiment-4

In [27]:
# Downloading CIFAR-10 dataset

from torchvision.datasets import CIFAR10

# Prepare the dataset
train_transforms = transforms.Compose([
    # transforms.RandomHorizontalFlip(),
    # transforms.RandomCrop(160, padding=8, padding_mode='reflect'),  # Data Augmentation
    transforms.CenterCrop(160),
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
    transforms.Grayscale()
])

test_transforms = transforms.Compose([
    transforms.CenterCrop(160),
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
    transforms.Grayscale()
])


train_dataset = CIFAR10("data/cifar/train/", train =True,  transform=train_transforms,download=True)

# Use 10% of the training set for validation
train_set_size = int(len(train_dataset) * 0.9)
val_set_size = len(train_dataset) - train_set_size

seed = torch.Generator().manual_seed(42)
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_set_size, val_set_size], generator=seed)
val_dataset.dataset.transform = test_transforms

# Use DataLoader to load the dataset
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, num_workers=8, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=128, num_workers=8, shuffle=False)

# Configure the test dataset
test_dataset = CIFAR10("data/cifar/test/", train =False,  transform=test_transforms,download=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, num_workers=8, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [28]:
# Model from scratch without fine tuning

model = CNNModelwithDropOut()

# Add EarlyStopping
early_stop_callback = EarlyStopping(monitor="val_loss",
                                    mode="min",
                                    patience=5)

# Configure Checkpoints
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    mode="min"
)

# Fit the model
trainer = L.Trainer(callbacks=[early_stop_callback, checkpoint_callback])
trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=val_loader)


## Restoring the best checkpoint
model = CNNModelwithDropOut.load_from_checkpoint(
            trainer.checkpoint_callback.best_model_path
        )
##Obtaining the test metrics
test_result = trainer.test(model, dataloaders=test_loader, verbose=False)

print(f"Best validation loss {trainer.checkpoint_callback.best_model_score.item()}\nTest loss: {test_result[0]['test_loss']}\nTest Accuracy: {test_result[0]['test_accuracy']}")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
d:\anaconda3\envs\cse6363\Lib\site-packages\lightning\pytorch\loops\utilities.py:72: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type               | Params
-------------------------------------------------
0 | estimator | Sequential         | 2.1 M 
1 | accuracy  | MulticlassAccuracy | 0     
-------------------------------------------------
2.1 M     Trainable params
0         Non-trainable params
2.1 M     Total params
8.470     Total estimated model params size (MB)


Epoch 69: 100%|██████████| 352/352 [01:29<00:00,  3.95it/s, v_num=16]      


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 79/79 [00:01<00:00, 43.88it/s]
Best validation loss 1.173889398574829
Test loss: 1.178741693496704
Test Accuracy: 0.5845999717712402


## Experiment-5

In [39]:
# Now let's import the weights obtained from training the same network on Imagenett dataset
best_model_path='d:\\UTA\\Sem-2\\ML\\CSE6363\\assignments\\assignment4\\lightning_logs\\version_15\\checkpoints\\epoch=8-step=603.ckpt'

model = CNNModelwithDropOut.load_from_checkpoint(
            best_model_path
        )

# Add EarlyStopping
early_stop_callback = EarlyStopping(monitor="val_loss",
                                    mode="min",
                                    patience=5)

# Configure Checkpoints
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    mode="min"
)

# Finetuning 
trainer = L.Trainer(callbacks=[early_stop_callback, checkpoint_callback])
trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=val_loader)

print(f"Restoring from {trainer.checkpoint_callback.best_model_path}")
## Restoring the best checkpoint
model = CNNModelwithDropOut.load_from_checkpoint(
            trainer.checkpoint_callback.best_model_path
        )
##Obtaining the test metrics
test_result = trainer.test(model, dataloaders=test_loader, verbose=False)

print(f"Best validation loss {trainer.checkpoint_callback.best_model_score.item()}\nTest loss: {test_result[0]['test_loss']}\nTest Accuracy: {test_result[0]['test_accuracy']}")


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
d:\anaconda3\envs\cse6363\Lib\site-packages\lightning\pytorch\loops\utilities.py:72: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type               | Params
-------------------------------------------------
0 | estimator | Sequential         | 2.1 M 
1 | accuracy  | MulticlassAccuracy | 0     
-------------------------------------------------
2.1 M     Trainable params
0         Non-trainable params
2.1 M     Total params
8.470     Total estimated model params size (MB)


Epoch 24: 100%|██████████| 352/352 [01:24<00:00,  4.16it/s, v_num=19]      
Restoring from d:\UTA\Sem-2\ML\CSE6363\assignments\assignment4\lightning_logs\version_19\checkpoints\epoch=19-step=7040.ckpt


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 79/79 [00:01<00:00, 51.91it/s]
Best validation loss 1.319576621055603
Test loss: 1.3074554204940796
Test Accuracy: 0.5364000201225281
